In [1]:
from time import time
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
from PIL import Image
from keras import optimizers
import keras.layers.advanced_activations
import scipy
import random
import matplotlib.pyplot as plt  
from scipy.stats import norm  
import keras
from keras.models import Sequential
from keras.initializers import VarianceScaling,RandomNormal
from keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, concatenate
from keras.layers import UpSampling2D, Reshape, Lambda, Flatten, Activation,Concatenate,Add
from keras.models import Model  
from keras.optimizers import SGD, Adadelta, Adagrad,Adam
from keras import backend as K  
from keras import objectives  
from keras.utils.vis_utils import plot_model  
from keras.utils import np_utils, generic_utils
import sys 
import tensorflow as tf
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.layers import BatchNormalization
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'
import sklearn
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
%matplotlib inline
from sklearn import manifold, datasets,cluster
from sklearn.utils import check_random_state
from sklearn import metrics

/home/hsun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_img = np.load("img_set_log.npy").reshape([-1,256,256,1])
train_seg = np.load("seg_set_log.npy").reshape([-1,256,256,1])
train_flo = np.load("flo_set_rgb_log.npy")
train_rflo = np.load("rflo_set_rgb_log.npy")
#train_img = np.log(train_img+1.0)
train_img = train_img/train_img.max()
train_seg = np_utils.to_categorical(train_seg, 2)

In [7]:
img_rows, img_cols = 256, 256
img_channels = 1

batch_size =50
latent_dim = 256
nb_epoch = 50
intermediate_dim =256
original_dim = img_rows*img_cols
LRelu = 'relu'

#USE = 'autoencoder'
#USE = 'vae'
#encoder:
input_img = Input(shape=(img_rows, img_cols,img_channels))
input_flo = Input(shape=(img_rows, img_cols,3))
input_rflo = Input(shape=(img_rows, img_cols,3))
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(input_img)
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
merge6 = Concatenate()([drop4,up6])
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = Concatenate()([conv3,up7])
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 =Concatenate()([conv2,up8])
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = Concatenate()([conv1,up9,input_flo,input_rflo])
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv10 = Conv2D(2, 1, activation = 'softmax')(conv9)

unet = Model(input = [input_img,input_flo,input_rflo], output = conv10)





#decoded = Conv2D(2, (3, 3), activation='softmax', padding='same')(upsample_10)
#decoded = Flatten()(decoded)
#decoded = Dense(256*256,activation='softmax')(decoded)


EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=0, mode='auto')
checkpoint = ModelCheckpoint('model_keras_unet.h5',monitor ='val_loss',verbose = 1,save_best_only = True)

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true[:,:,:,1]) 
    y_pred_f = K.flatten(y_pred[:,:,:,1]) 
    intersection = K.sum(y_true_f * y_pred_f) 
    return (2. * intersection + 1e-6) / (K.sum(K.square(y_true_f)) + K.sum(K.square(y_pred_f)) + 1e-6)
def dice_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

#def ae_loss(x, decoded):  
#    xent_loss = original_dim * objectives.mean_squared_error(x,decoded)
#    return xent_loss
adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#autoencoder = Model(inputs=input_img, outputs=decoded)
unet.compile(optimizer=adam, loss=dice_loss)

/home/hsun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:57: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("co...)`


In [9]:
unet.fit([train_img[:1150],train_flo[:1150],train_rflo[:1150]], train_seg[:1150],
        shuffle=True,
        epochs=200,
        batch_size=20,
        validation_data=([train_img[1150:],train_flo[1150:],train_rflo[1150:]],train_seg[1150:]),callbacks=[EarlyStopping,checkpoint])

Train on 1150 samples, validate on 127 samples
Epoch 1/200
1150/1150 [==============================] - 81s 70ms/step - loss: -0.2940 - val_loss: -0.2080

Epoch 00001: val_loss improved from inf to -0.20804, saving model to model_keras_unet.h5
Epoch 2/200
1150/1150 [==============================] - 73s 63ms/step - loss: -0.4243 - val_loss: -0.2540

Epoch 00002: val_loss improved from -0.20804 to -0.25405, saving model to model_keras_unet.h5
Epoch 3/200
1150/1150 [==============================] - 73s 63ms/step - loss: -0.4367 - val_loss: -0.0400

Epoch 00003: val_loss did not improve from -0.25405
Epoch 4/200
1150/1150 [==============================] - 73s 63ms/step - loss: -0.4358 - val_loss: -0.2582

Epoch 00004: val_loss improved from -0.25405 to -0.25819, saving model to model_keras_unet.h5
Epoch 5/200
1150/1150 [==============================] - 72s 63ms/step - loss: -0.4504 - val_loss: -0.3522

Epoch 00005: val_loss improved from -0.25819 to -0.35216, saving model to model_kera

# mid

In [3]:
img_rows, img_cols = 256, 256
img_channels = 1

batch_size =50
latent_dim = 256
nb_epoch = 50
intermediate_dim =256
original_dim = img_rows*img_cols
LRelu = 'relu'

#USE = 'autoencoder'
#USE = 'vae'
#encoder:
input_img = Input(shape=(img_rows, img_cols,img_channels))
input_flo = Input(shape=(img_rows, img_cols,3))
input_rflo = Input(shape=(img_rows, img_cols,3))

concate_flo = Concatenate()([input_flo,input_rflo])
conv_flo_1 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(concate_flo)
conv_flo_1 = Activation('relu')(conv_flo_1)
conv_flo_1 = BatchNormalization()(conv_flo_1)
maxpool_flo_1 = MaxPooling2D((2, 2),padding='same')(conv_flo_1)
conv_flo_2 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(maxpool_flo_1)
conv_flo_2 = Activation('relu')(conv_flo_2)
conv_flo_2 = BatchNormalization()(conv_flo_2)
maxpool_flo_2 = MaxPooling2D((2, 2),  padding='same')(conv_flo_2)
conv_flo_3 = Conv2D(20, (3, 3),padding='same',kernel_initializer='normal',dilation_rate=1)(maxpool_flo_2)
conv_flo_3 = Activation(LRelu)(conv_flo_3)
conv_flo_3 = BatchNormalization()(conv_flo_3)
maxpool_flo_3 = MaxPooling2D((2, 2),  padding='same')(conv_flo_3)
conv_flo_4 = Conv2D(20, (3, 3),padding='same',kernel_initializer='normal',dilation_rate=1)(maxpool_flo_3)
conv_flo_4 = Activation(LRelu)(conv_flo_4)
conv_flo_4 = BatchNormalization()(conv_flo_4)
maxpool_flo_4 = MaxPooling2D((2, 2),  padding='same')(conv_flo_4)


conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(input_img)
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

drop5 = Concatenate()([maxpool_flo_4,drop5])

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
merge6 = Concatenate()([drop4,up6])
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = Concatenate()([conv3,up7])
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 =Concatenate()([conv2,up8])
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = Concatenate()([conv1,up9])
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv10 = Conv2D(2, 1, activation = 'softmax')(conv9)

unet = Model(input = [input_img,input_flo,input_rflo], output = conv10)





#decoded = Conv2D(2, (3, 3), activation='softmax', padding='same')(upsample_10)
#decoded = Flatten()(decoded)
#decoded = Dense(256*256,activation='softmax')(decoded)


EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=0, mode='auto')
checkpoint = ModelCheckpoint('model_keras_unet_mid.h5',monitor ='val_loss',verbose = 1,save_best_only = True)

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true[:,:,:,1]) 
    y_pred_f = K.flatten(y_pred[:,:,:,1]) 
    intersection = K.sum(y_true_f * y_pred_f) 
    return (2. * intersection + 1e-6) / (K.sum(K.square(y_true_f)) + K.sum(K.square(y_pred_f)) + 1e-6)
def dice_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

#def ae_loss(x, decoded):  
#    xent_loss = original_dim * objectives.mean_squared_error(x,decoded)
#    return xent_loss
adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#autoencoder = Model(inputs=input_img, outputs=decoded)
unet.compile(optimizer=adam, loss=dice_loss)

/home/hsun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:79: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("co...)`


In [4]:
unet.fit([train_img[:1150],train_flo[:1150],train_rflo[:1150]], train_seg[:1150],
        shuffle=True,
        epochs=200,
        batch_size=20,
        validation_data=([train_img[1150:],train_flo[1150:],train_rflo[1150:]],train_seg[1150:]),callbacks=[EarlyStopping,checkpoint])

Train on 1150 samples, validate on 127 samples
Epoch 1/200
1150/1150 [==============================] - 89s 77ms/step - loss: -0.3256 - val_loss: -0.1619

Epoch 00001: val_loss improved from inf to -0.16193, saving model to model_keras_unet_mid.h5
Epoch 2/200
1150/1150 [==============================] - 74s 64ms/step - loss: -0.4288 - val_loss: -0.3183

Epoch 00002: val_loss improved from -0.16193 to -0.31830, saving model to model_keras_unet_mid.h5
Epoch 3/200
1150/1150 [==============================] - 74s 64ms/step - loss: -0.4757 - val_loss: -0.3378

Epoch 00003: val_loss improved from -0.31830 to -0.33775, saving model to model_keras_unet_mid.h5
Epoch 4/200
1150/1150 [==============================] - 74s 64ms/step - loss: -0.5107 - val_loss: -0.3259

Epoch 00004: val_loss did not improve from -0.33775
Epoch 5/200
1150/1150 [==============================] - 74s 64ms/step - loss: -0.5255 - val_loss: -0.4304

Epoch 00005: val_loss improved from -0.33775 to -0.43038, saving model t

1150/1150 [==============================] - 73s 64ms/step - loss: -0.8714 - val_loss: -0.7628

Epoch 00045: val_loss did not improve from -0.76349
Epoch 46/200
1150/1150 [==============================] - 74s 64ms/step - loss: -0.8723 - val_loss: -0.7469

Epoch 00046: val_loss did not improve from -0.76349
Epoch 47/200
1150/1150 [==============================] - 74s 64ms/step - loss: -0.8761 - val_loss: -0.7645

Epoch 00047: val_loss improved from -0.76349 to -0.76452, saving model to model_keras_unet_mid.h5
Epoch 48/200
1150/1150 [==============================] - 73s 64ms/step - loss: -0.8801 - val_loss: -0.7661

Epoch 00048: val_loss improved from -0.76452 to -0.76608, saving model to model_keras_unet_mid.h5
Epoch 49/200
1150/1150 [==============================] - 73s 64ms/step - loss: -0.8825 - val_loss: -0.7757

Epoch 00049: val_loss improved from -0.76608 to -0.77572, saving model to model_keras_unet_mid.h5
Epoch 50/200
1150/1150 [==============================] - 74s 64ms/ste

# conv last

In [3]:
img_rows, img_cols = 256, 256
img_channels = 1

batch_size =50
latent_dim = 256
nb_epoch = 50
intermediate_dim =256
original_dim = img_rows*img_cols
LRelu = 'relu'

#USE = 'autoencoder'
#USE = 'vae'
#encoder:
input_img = Input(shape=(img_rows, img_cols,img_channels))
input_flo = Input(shape=(img_rows, img_cols,3))
input_rflo = Input(shape=(img_rows, img_cols,3))

concate_flo = Concatenate()([input_flo,input_rflo])
conv_flo_1 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(concate_flo)
conv_flo_1 = Activation('relu')(conv_flo_1)
conv_flo_1 = BatchNormalization()(conv_flo_1)
#maxpool_flo_1 = MaxPooling2D((2, 2),padding='same')(conv_flo_1)
conv_flo_2 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(conv_flo_1)
conv_flo_2 = Activation('relu')(conv_flo_2)
conv_flo_2 = BatchNormalization()(conv_flo_2)
#maxpool_flo_2 = MaxPooling2D((2, 2),  padding='same')(conv_flo_2)
conv_flo_3 = Conv2D(20, (3, 3),padding='same',kernel_initializer='normal',dilation_rate=1)(conv_flo_2)
conv_flo_3 = Activation(LRelu)(conv_flo_3)
conv_flo_3 = BatchNormalization()(conv_flo_3)



conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(input_img)
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

#drop5 = Concatenate()([maxpool_flo_4,drop5])

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
merge6 = Concatenate()([drop4,up6])
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = Concatenate()([conv3,up7])
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 =Concatenate()([conv2,up8])
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = Concatenate()([conv1,up9,conv_flo_3])
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv10 = Conv2D(2, 1, activation = 'softmax')(conv9)

unet = Model(input = [input_img,input_flo,input_rflo], output = conv10)





#decoded = Conv2D(2, (3, 3), activation='softmax', padding='same')(upsample_10)
#decoded = Flatten()(decoded)
#decoded = Dense(256*256,activation='softmax')(decoded)


EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=0, mode='auto')
checkpoint = ModelCheckpoint('model_keras_unet.h5',monitor ='val_loss',verbose = 1,save_best_only = True)

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true[:,:,:,1]) 
    y_pred_f = K.flatten(y_pred[:,:,:,1]) 
    intersection = K.sum(y_true_f * y_pred_f) 
    return (2. * intersection + 1e-6) / (K.sum(K.square(y_true_f)) + K.sum(K.square(y_pred_f)) + 1e-6)
def dice_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

#def ae_loss(x, decoded):  
#    xent_loss = original_dim * objectives.mean_squared_error(x,decoded)
#    return xent_loss
adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#autoencoder = Model(inputs=input_img, outputs=decoded)
unet.compile(optimizer=adam, loss=dice_loss)


/home/hsun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:75: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("co...)`


In [5]:
unet.fit([train_img[:1150],train_flo[:1150],train_rflo[:1150]], train_seg[:1150],
        shuffle=True,
        epochs=200,
        batch_size=20,
        validation_data=([train_img[1150:],train_flo[1150:],train_rflo[1150:]],train_seg[1150:]),callbacks=[EarlyStopping,checkpoint])

Train on 1150 samples, validate on 127 samples
Epoch 1/200
1150/1150 [==============================] - 76s 66ms/step - loss: -0.9381 - val_loss: -0.8167

Epoch 00001: val_loss did not improve from -0.82090
Epoch 2/200
1150/1150 [==============================] - 79s 68ms/step - loss: -0.9238 - val_loss: -0.8172

Epoch 00002: val_loss did not improve from -0.82090
Epoch 3/200
1150/1150 [==============================] - 79s 68ms/step - loss: -0.9410 - val_loss: -0.8058

Epoch 00003: val_loss did not improve from -0.82090
Epoch 4/200
1150/1150 [==============================] - 79s 68ms/step - loss: -0.9469 - val_loss: -0.8269

Epoch 00004: val_loss improved from -0.82090 to -0.82685, saving model to model_keras_unet.h5
Epoch 5/200
1150/1150 [==============================] - 78s 68ms/step - loss: -0.9525 - val_loss: -0.8122

Epoch 00005: val_loss did not improve from -0.82685
Epoch 6/200
1150/1150 [==============================] - 78s 68ms/step - loss: -0.9556 - val_loss: -0.8247

Epo

In [ ]:
0.8425

# 跑一个不用flo的baseline

In [5]:
img_rows, img_cols = 256, 256
img_channels = 1

batch_size =50
latent_dim = 256
nb_epoch = 50
intermediate_dim =256
original_dim = img_rows*img_cols
LRelu = 'relu'

#USE = 'autoencoder'
#USE = 'vae'
#encoder:
input_img = Input(shape=(img_rows, img_cols,img_channels))

conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(input_img)
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
merge6 = Concatenate()([drop4,up6])
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = Concatenate()([conv3,up7])
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 =Concatenate()([conv2,up8])
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = Concatenate()([conv1,up9])
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv10 = Conv2D(2, 1, activation = 'softmax')(conv9)

unet = Model(input = input_img, output = conv10)





#decoded = Conv2D(2, (3, 3), activation='softmax', padding='same')(upsample_10)
#decoded = Flatten()(decoded)
#decoded = Dense(256*256,activation='softmax')(decoded)


EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=0, mode='auto')
checkpoint = ModelCheckpoint('model_keras_unet.h5',monitor ='val_loss',verbose = 1,save_best_only = True)

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true[:,:,:,1]) 
    y_pred_f = K.flatten(y_pred[:,:,:,1]) 
    intersection = K.sum(y_true_f * y_pred_f) 
    return (2. * intersection + 1e-6) / (K.sum(K.square(y_true_f)) + K.sum(K.square(y_pred_f)) + 1e-6)
def dice_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

#def ae_loss(x, decoded):  
#    xent_loss = original_dim * objectives.mean_squared_error(x,decoded)
#    return xent_loss
adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#autoencoder = Model(inputs=input_img, outputs=decoded)
unet.compile(optimizer=adam, loss=dice_loss)

/home/hsun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [6]:
unet.fit(train_img[:1150], train_seg[:1150],
        shuffle=True,
        epochs=200,
        batch_size=20,
        validation_data=(train_img[1150:],train_seg[1150:]),callbacks=[EarlyStopping,checkpoint])

Train on 1150 samples, validate on 127 samples
Epoch 1/200
1150/1150 [==============================] - 73s 64ms/step - loss: -0.3209 - val_loss: -0.0034

Epoch 00001: val_loss improved from inf to -0.00336, saving model to model_keras_unet.h5
Epoch 2/200
1150/1150 [==============================] - 71s 61ms/step - loss: -0.4366 - val_loss: -0.3590

Epoch 00002: val_loss improved from -0.00336 to -0.35899, saving model to model_keras_unet.h5
Epoch 3/200
1150/1150 [==============================] - 70s 61ms/step - loss: -0.4729 - val_loss: -0.3981

Epoch 00003: val_loss improved from -0.35899 to -0.39809, saving model to model_keras_unet.h5
Epoch 4/200
1150/1150 [==============================] - 70s 61ms/step - loss: -0.4725 - val_loss: -0.3908

Epoch 00004: val_loss did not improve from -0.39809
Epoch 5/200
1150/1150 [==============================] - 70s 61ms/step - loss: -0.5092 - val_loss: -0.3845

Epoch 00005: val_loss did not improve from -0.39809
Epoch 6/200
1150/1150 [=========

1150/1150 [==============================] - 70s 61ms/step - loss: -0.9676 - val_loss: -0.8002

Epoch 00047: val_loss did not improve from -0.82821
Epoch 48/200
1150/1150 [==============================] - 70s 61ms/step - loss: -0.9696 - val_loss: -0.8180

Epoch 00048: val_loss did not improve from -0.82821
Epoch 49/200
1150/1150 [==============================] - 70s 61ms/step - loss: -0.9692 - val_loss: -0.8382

Epoch 00049: val_loss improved from -0.82821 to -0.83823, saving model to model_keras_unet.h5
Epoch 50/200
1150/1150 [==============================] - 70s 61ms/step - loss: -0.9722 - val_loss: -0.8327

Epoch 00050: val_loss did not improve from -0.83823
Epoch 51/200
1150/1150 [==============================] - 70s 61ms/step - loss: -0.9735 - val_loss: -0.8117

Epoch 00051: val_loss did not improve from -0.83823
Epoch 52/200
1150/1150 [==============================] - 70s 61ms/step - loss: -0.9747 - val_loss: -0.8246

Epoch 00052: val_loss did not improve from -0.83823
Epoch

# 不用flo 加上Augmentation

In [5]:
img_rows, img_cols = 256, 256
img_channels = 1

batch_size =20
latent_dim = 256
nb_epoch = 50
intermediate_dim =256
original_dim = img_rows*img_cols
LRelu = 'relu'

#USE = 'autoencoder'
#USE = 'vae'
#encoder:
input_img = Input(shape=(img_rows, img_cols,img_channels))

conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(input_img)
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
merge6 = Concatenate()([drop4,up6])
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = Concatenate()([conv3,up7])
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 =Concatenate()([conv2,up8])
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = Concatenate()([conv1,up9])
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv10 = Conv2D(2, 1, activation = 'softmax')(conv9)

unet = Model(input = input_img, output = conv10)





#decoded = Conv2D(2, (3, 3), activation='softmax', padding='same')(upsample_10)
#decoded = Flatten()(decoded)
#decoded = Dense(256*256,activation='softmax')(decoded)


EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=0, mode='auto')
checkpoint = ModelCheckpoint('model_keras_unet.h5',monitor ='val_loss',verbose = 1,save_best_only = True)

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true[:,:,:,1]) 
    y_pred_f = K.flatten(y_pred[:,:,:,1]) 
    intersection = K.sum(y_true_f * y_pred_f) 
    return (2. * intersection + 1e-6) / (K.sum(K.square(y_true_f)) + K.sum(K.square(y_pred_f)) + 1e-6)
def dice_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

#def ae_loss(x, decoded):  
#    xent_loss = original_dim * objectives.mean_squared_error(x,decoded)
#    return xent_loss
adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#autoencoder = Model(inputs=input_img, outputs=decoded)
unet.compile(optimizer=adam, loss=dice_loss)

/home/hsun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [6]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=0, mode='auto')
csv_logger = CSVLogger('Unet_logdata_loss.csv')
print("using data augmentation")
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
datagen.fit(train_img[:1150])

# Fit the model on the batches generated by datagen.flow().
unet.fit_generator(datagen.flow(train_img[:1150], train_seg[:1150], batch_size=batch_size),
                    steps_per_epoch=train_img[:1150].shape[0] // batch_size,
                    validation_data=(train_img[1150:],train_seg[1150:]),
                    epochs=200, verbose=1, max_q_size=100,
                    callbacks=[lr_reducer, early_stopper, csv_logger])


using data augmentation


/home/hsun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=57, validation_data=(array([[[..., epochs=200, verbose=1, callbacks=[<keras.ca..., max_queue_size=100)`


Epoch 1/200
57/57 [==============================] - 72s 1s/step - loss: -0.3119 - val_loss: -0.0125
Epoch 2/200
57/57 [==============================] - 70s 1s/step - loss: -0.4292 - val_loss: -0.2413
Epoch 3/200
57/57 [==============================] - 69s 1s/step - loss: -0.4387 - val_loss: -0.2535
Epoch 4/200
57/57 [==============================] - 68s 1s/step - loss: -0.4365 - val_loss: -0.2842
Epoch 5/200
57/57 [==============================] - 68s 1s/step - loss: -0.4406 - val_loss: -0.2639
Epoch 6/200
57/57 [==============================] - 68s 1s/step - loss: -0.4408 - val_loss: -0.2436
Epoch 7/200
57/57 [==============================] - 68s 1s/step - loss: -0.4494 - val_loss: -0.2822
Epoch 8/200
57/57 [==============================] - 68s 1s/step - loss: -0.4418 - val_loss: -0.3193
Epoch 9/200
57/57 [==============================] - 68s 1s/step - loss: -0.4516 - val_loss: -0.2180
Epoch 10/200
57/57 [==============================] - 68s 1s/step - loss: -0.4579 - val_los